# Pythia SSYM/S461 NMI-style

In [115]:
import pandas
import numpy
from copy import deepcopy
import numpy
import math
from sklearn.model_selection import RepeatedStratifiedKFold, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

## Prepare SSYM data

In [116]:
pythia_ssym_raw = pandas.read_csv('ssym_Pythia_MB_dir_inv.csv')
pythia_ssym_raw.head()

,name,ddg,pythiascore,name_inv,ddg_inv,pythiascore_inv,mb,pythiascore+mb,pythiascore_inv-mb
0,1AMQ_C_180_Y,-2.3,-12.20,1QIR_Y_180_C,2.3,-1.12,0.466740,-1.300145,-0.628945
1,1AMQ_C_180_F,-1.6,-10.53,1QIS_F_180_C,1.6,-0.28,0.171029,-1.353995,-0.211581
2,1AMQ_C_180_W,-3.9,-14.09,1QIT_W_180_C,3.9,0.02,0.784479,-1.256128,-0.781582
3,1AMQ_C_180_S,-1.9,-3.65,5EAA_S_180_C,1.9,3.72,-0.823947,-1.352564,1.362702
4,1BNI_F_5_L,-4.1,-0.17,1BRG_L_5_F,4.1,-6.17,-0.117133,-0.141753,-0.776447


In [117]:
_dir=pythia_ssym_raw[['name','ddg','pythiascore']]
#[_dir.head()
_inv=pythia_ssym_raw[['name_inv','ddg_inv','pythiascore_inv']].rename({'name_inv': 'name', 'ddg_inv': 'ddg','pythiascore_inv':'pythiascore' }, axis=1)
#_inv.head()

In [118]:
pythia_ssym_df= pandas.concat([_dir,_inv],ignore_index=True)
assert len(pythia_ssym_df) == 2*len(pythia_ssym_raw)
pythia_ssym_df

,name,ddg,pythiascore
0,1AMQ_C_180_Y,-2.3,-12.20
1,1AMQ_C_180_F,-1.6,-10.53
2,1AMQ_C_180_W,-3.9,-14.09
3,1AMQ_C_180_S,-1.9,-3.65
4,1BNI_F_5_L,-4.1,-0.17
...,...,...,...
679,1BTI_A_22_F,1.2,-10.07
680,1BPT_A_23_Y,5.9,13.48
681,8PTI_G_35_Y,5.0,-19.10
682,1NAG_G_43_N,5.7,13.90


In [119]:
pythia_ssym_df['from_aa']=pythia_ssym_df.apply(lambda x: x['name'].split('_')[1],axis=1)
pythia_ssym_df['to_aa']=pythia_ssym_df.apply(lambda x: x['name'][-1],axis=1)
pythia_ssym_df.head()

,name,ddg,pythiascore,from_aa,to_aa
0,1AMQ_C_180_Y,-2.3,-12.20,C,Y
1,1AMQ_C_180_F,-1.6,-10.53,C,F
2,1AMQ_C_180_W,-3.9,-14.09,C,W
3,1AMQ_C_180_S,-1.9,-3.65,C,S
4,1BNI_F_5_L,-4.1,-0.17,F,L


## Rose/ddMBC fit

In [120]:
rose_df=pandas.read_csv('rose1985.csv',index_col='Parameter')
rose_df

,Rose1985
Parameter,
A,86.6
C,132.3
D,97.8
E,113.9
F,194.1
G,62.9
H,155.8
I,158.0
K,115.5


In [121]:
# esmif_df.head()
def rose_delta(x):
    return rose_df["Rose1985"][x["from_aa"]] - rose_df["Rose1985"][x["to_aa"]]

In [122]:
pythia_ssym_df['rose_delta'] = pythia_ssym_df.apply(rose_delta, 
               axis=1
              )
pythia_ssym_df

,name,ddg,pythiascore,from_aa,to_aa,rose_delta
0,1AMQ_C_180_Y,-2.3,-12.20,C,Y,-45.4
1,1AMQ_C_180_F,-1.6,-10.53,C,F,-61.8
2,1AMQ_C_180_W,-3.9,-14.09,C,W,-92.3
3,1AMQ_C_180_S,-1.9,-3.65,C,S,46.7
4,1BNI_F_5_L,-4.1,-0.17,F,L,30.0
...,...,...,...,...,...,...
679,1BTI_A_22_F,1.2,-10.07,A,F,-107.5
680,1BPT_A_23_Y,5.9,13.48,A,Y,-91.1
681,8PTI_G_35_Y,5.0,-19.10,G,Y,-114.8
682,1NAG_G_43_N,5.7,13.90,G,N,-40.4


### fit Rose model

In [123]:
X_ssym = pythia_ssym_df[['pythiascore','rose_delta']].to_numpy()
y = pythia_ssym_df['ddg'].to_numpy()
rose_model = LinearRegression(fit_intercept=False).fit(X_ssym,y)

In [124]:
rose_model.coef_ , math.sqrt(rose_model.score(X_ssym,y))

(array([ 0.14653462, -0.01177167]), 0.6755002670651183)

In [125]:
pythia_ssym_df['ddmbc_rose']=rose_model.predict(X_ssym)
pythia_ssym_df.head()

,name,ddg,pythiascore,from_aa,to_aa,rose_delta,ddmbc_rose
0,1AMQ_C_180_Y,-2.3,-12.20,C,Y,-45.4,-1.253289
1,1AMQ_C_180_F,-1.6,-10.53,C,F,-61.8,-0.815520
2,1AMQ_C_180_W,-3.9,-14.09,C,W,-92.3,-0.978148
3,1AMQ_C_180_S,-1.9,-3.65,C,S,46.7,-1.084588
4,1BNI_F_5_L,-4.1,-0.17,F,L,30.0,-0.378061


#### fit stats

In [126]:
pythia_ssym_df[['ddg','pythiascore','ddmbc_rose']].corr()

,ddg,pythiascore,ddmbc_rose
ddg,1.000000,0.649582,0.701653
pythiascore,0.649582,1.000000,0.903809
ddmbc_rose,0.701653,0.903809,1.000000


In [127]:
for _ in ["pythiascore", "ddmbc_rose"]:
    print(
        _,
        "ssym RMSE:",
        math.sqrt(mean_squared_error(pythia_ssym_df["ddg"], pythia_ssym_df[_])),
    )

pythiascore ssym RMSE: 6.242522368907924
ddmbc_rose ssym RMSE: 1.3663891060364857


### Crossvalidate

In [128]:
fit_rsq = cross_val_score(
    rose_model,
    X_ssym,
    y,
    cv=RepeatedStratifiedKFold(
        random_state=2411,
        n_splits=5,
        n_repeats=10,
        # shuffle=True,
    ).split(X_ssym, y > 0),
)
# fit_rsq

_r = numpy.sqrt(fit_rsq)
print("CV:", _r.mean(), "+-", _r.std())

CV: 0.6675212862920354 +- 0.04629125470948791


# S461

In [129]:
def go_to_my_format(x):
    for _v, _k in zip(
        x["name"].split("_"), ["pdb_code", "from_aa", "position", "to_aa"]
    ):
        x[_k] = _v
    return x

In [130]:
esmif_df=esmif_df.apply(lambda x: go_to_my_format(x), axis=1)
esmif_df

NameError: name 'esmif_df' is not defined

In [ ]:
optfep_rsq=cross_val_score(LinearRegression(fit_intercept=True),
                   X_optfep,y,
                   cv=RepeatedStratifiedKFold(n_splits=5,
                                              n_repeats=10,
                                              #shuffle=True,
                                             ).split(X_optfep, y > 0))
optfep_rsq

In [ ]:
numpy.sqrt(optfep_rsq).mean(), numpy.sqrt(optfep_rsq).std()

## Get S669

In [131]:
def go_to_my_format(x):
    for _v, _k in zip(
        x["name"].split("_"), ["pdb_code", "from_aa", "position", "to_aa"]
    ):
        x[_k] = _v
    return x

In [132]:
esmif_s669_df = (
    pandas.read_csv("../Pythia/pythia_s669_augd.csv", index_col=None)
    .rename(str.lower, axis=1)
    .apply(lambda x: go_to_my_format(x), axis=1)
)
#esmif_s669_df['experimental']=True
esmif_s669_df

,name,ddg,pythiascore,pdb_code,from_aa,position,to_aa
0,1a0f_S_11_A,-1.800,-2.911759,1a0f,S,11,A
1,1a7v_A_104_H,-2.690,-8.782735,1a7v,A,104,H
2,1a7v_K_13_H,-0.600,-4.157711,1a7v,K,13,H
3,1a7v_K_20_H,-2.880,-8.267227,1a7v,K,20,H
4,1a7v_D_3_H,-1.360,-12.814983,1a7v,D,3,H
...,...,...,...,...,...,...,...
664,5jxb_D_25_P,-1.440,-5.337091,5jxb,D,25,P
665,5oaq_Y_199_H,-2.990,-13.397189,5oaq,Y,199,H
666,5vp3_S_128_G,-0.378,-8.731921,5vp3,S,128,G
667,5vp3_V_183_T,0.354,-3.005847,5vp3,V,183,T


## S461 

In [133]:
#esmif_s669_sym_df[esmif_s669_sym_df['s461']==True][['ddg','ddfep_opt','pythiascore']].corr()

In [134]:
s461_df=pandas.read_csv('../Pythia/S461.mut',sep=' ')

In [135]:
## extract mutations to simpler csv
s461_mutations = pandas.DataFrame(s461_df['ddg'])
s461_mutations['pdb_code'] = s461_df['pdb'].apply(lambda x: x[:4].lower())
s461_mutations['chain'] = s461_df['pdb'].apply(lambda x: x[-1].upper())
s461_mutations['position'] = s461_df['mut'].apply(lambda x: int(x[1:-1]))
s461_mutations['from_aa'] = s461_df['mut'].apply(lambda x: x[0].upper())
s461_mutations['to_aa'] = s461_df['mut'].apply(lambda x: x[-1].upper())
s461_mutations['name']= s461_mutations.apply(lambda x: '_'.join([ x['pdb_code'],
                                                          x['from_aa'],
                                                          str(x['position']),
                                                          x['to_aa'],
                                                         ]), axis=1)
s461_mutations

,ddg,pdb_code,chain,position,from_aa,to_aa,name
0,-1.800,1a0f,A,11,S,A,1a0f_S_11_A
1,-1.745,1ba3,A,461,H,D,1ba3_H_461_D
2,0.287,1ba3,A,489,H,D,1ba3_H_489_D
3,-0.287,1ba3,A,489,H,K,1ba3_H_489_K
4,-0.263,1ba3,A,489,H,M,1ba3_H_489_M
...,...,...,...,...,...,...,...
456,0.060,4he7,A,19,A,G,4he7_A_19_G
457,-0.460,4he7,A,19,A,K,4he7_A_19_K
458,-1.440,5jxb,A,329,D,G,5jxb_D_329_G
459,-1.440,5jxb,A,329,D,P,5jxb_D_329_P


In [136]:
esmif_s669_df['s461']=esmif_s669_df.apply(lambda x: x['pdb_code'] in list(s461_mutations['pdb_code']), axis=1)

In [137]:
#esmif_s669_sym_df['s461']=esmif_s669_sym_df.apply(lambda x: mut_hash(x) in s461_hashes,axis=1)
esmif_s461_df= deepcopy(esmif_s669_df[esmif_s669_df['s461'] == True])
esmif_s461_df['rose_delta']= esmif_s461_df.apply(rose_delta,axis=1)
esmif_s461_df['ddmbc_rose']=rose_model.predict(esmif_s461_df[['ddg','rose_delta']].to_numpy())
esmif_s461_df.head()

,name,ddg,pythiascore,pdb_code,from_aa,position,to_aa,s461,rose_delta,ddmbc_rose
0,1a0f_S_11_A,-1.800,-2.911759,1a0f,S,11,A,True,-1.0,-0.251991
14,1ba3_H_457_D,-1.745,-7.294972,1ba3,H,457,D,True,58.0,-0.938460
15,1ba3_H_485_D,0.287,3.924647,1ba3,H,485,D,True,58.0,-0.640701
16,1ba3_H_485_K,-0.287,5.853164,1ba3,H,485,K,True,40.3,-0.516454
17,1ba3_H_485_M,-0.263,-3.824930,1ba3,H,485,M,True,-17.1,0.162757


In [138]:
pythia_corr=esmif_s461_df[['ddg','ddmbc_rose','pythiascore']].corr()
pythia_rmse=math.sqrt(mean_squared_error(esmif_s461_df['ddg'],esmif_s461_df['pythiascore']))
rose_rmse =math.sqrt(mean_squared_error(esmif_s461_df['ddg'],esmif_s461_df['ddmbc_rose']))
print('RMSE(ddmbc_rose):',rose_rmse)     
print('RMSE(pythiascore):', pythia_rmse)
display(pythia_corr)

RMSE(ddmbc_rose): 1.2168553584918722
RMSE(pythiascore): 7.390213195496487


,ddg,ddmbc_rose,pythiascore
ddg,1.000000,0.616999,0.425031
ddmbc_rose,0.616999,1.000000,0.151263
pythiascore,0.425031,0.151263,1.000000


## CSV

In [139]:
pythia_nmi={}
pythia_nmi['plain_pearson']= pythia_corr.at['ddg','pythiascore']
pythia_nmi['test_pearson']= pythia_corr.at['ddg','ddmbc_rose'] 
pythia_nmi['delta_pearson']= pythia_nmi['test_pearson']-pythia_nmi['plain_pearson']
pythia_nmi['plain_RMSE']= pythia_rmse
pythia_nmi['test_RMSE']= rose_rmse
pythia_nmi['delta_pearson']= pythia_nmi['test_pearson']-pythia_nmi['plain_pearson']
pythia_nmi['delta_RMSE']= pythia_nmi['test_RMSE']-pythia_nmi['plain_RMSE']
pythia_nmi['type']= 'struct.PLM'
pythia_nmi['mass_balance']= 'n'
pythia_nmi_df=pandas.DataFrame(pythia_nmi, index=['pythia'])
pythia_nmi_df

,plain_pearson,test_pearson,delta_pearson,plain_RMSE,test_RMSE,delta_RMSE,type,mass_balance
pythia,0.425031,0.616999,0.191968,7.390213,1.216855,-6.173358,struct.PLM,n


In [142]:
pythia_nmi_df.to_csv('pythia_nmi.csv',index_label=None)